#AutoML(Automated Machine Learning)

사용자가 직접 모델을 설계·튜닝하지 않아도, 데이터 전처리 → 모델 선택 → 하이퍼파라미터 최적화까지 **자동으로 수행**하는 기술.

| 단계 | 설명 | 예시 |
|------|------|------|
| 데이터 전처리 | 결측치 처리, 스케일링, 인코딩 | `StandardScaler`, `OneHotEncoder` |
| 모델 탐색 | 여러 알고리즘 비교·평가 | `RandomForest`, `XGBoost`, `LightGBM`, `CatBoost`, ... |
| 하이퍼파라미터 탐색 | 각 모델의 설정값 자동 최적화 | `max_depth`, `learning_rate`, `n_estimators` 등 |
| 평가 및 선택 | 교차검증 성능 비교 후 최적 모델 선택 | RMSE, MAE, R² 등 |

---

- 머신러닝 모델의 성능은 **하이퍼파라미터 설정에 매우 민감**하다.  
- 수작업으로 모든 조합을 시도하기엔 **시간·자원 낭비**가 크다.  
- AutoML은 **탐색과 최적화 과정을 자동화**하여 연구자나 실무자가 **데이터 해석과 의사결정**에 집중할 수 있도록 돕는다.


# iris_data를 xgboost로 적용을 해보자.

In [ ]:
#import packages
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris

#Loading iris dataset from sklearn
iris = load_iris()

#independent feautres
X = iris.data

# target features
y = iris.target

In [ ]:
#import XGboost
from xgboost import XGBClassifier

#Defining XGB Classification model
clf = XGBClassifier()

- xgboost의 단점은 복잡한 하이퍼 파라미터가 있다.

# 1.Grid SearchCV

---

사용자가 지정한 하이퍼파라미터 후보값들을 **모든 조합으로 탐색(Grid Search)** 하여,  가장 좋은 성능을 내는 파라미터를 찾는 방법이다.

즉, 각 하이퍼파라미터에 대해 가능한 값을 리스트로 입력하면, 모든 가능한 조합을 반복적으로 학습·평가하여 **최적의 하이퍼파라미터 조합**을 자동으로 찾아준다.

---

**주요 특징**

| 항목 | 설명 |
|------|------|
| **탐색 방식** | 모든 하이퍼파라미터 조합을 완전 탐색 |
| **평가 방법** | 교차검증(Cross Validation)을 이용해 각 조합의 평균 성능 계산 |
| **장점** | 단순하고 구현이 쉬움 |
| **단점** | 조합 수가 많아질수록 계산 비용이 급격히 증가 |

In [ ]:
#Importing packages from sklearn

from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics

#defining a set of values as a dictionary for hyperparameters

param_grid = {
    "n_estimators":[100,200,300,400],
    "max_depth":[1,3,5,7],
    "reg_lambda":[.01,.1,.5]
}

#declaring GridSearchCV model

model = model_selection.GridSearchCV(
    estimator = clf,
    param_grid = param_grid,
    scoring = 'accuracy',
    verbose = 10,
    n_jobs = 1,
    cv = 5
)

#fitting values to the gridsearchcv model

model.fit(X,y)
#printing the best possible values to enhance accuracy
print(model.best_params_)
print(model.best_estimator_)
#printing the best score
print(model.best_score_)
#GridSearchCV model save
# import joblib
# joblib.dump(model, 'model_xgboost.pkl')
# joblib.load('model_xgboost.pkl')

# 2.RandomizedSearchCV

---
그리드 서치처럼 모든 조합을 탐색하지 않고, **지정된 하이퍼파라미터 공간에서 무작위로 일부 조합만 선택**하여 탐색하는 방식이다.

하이퍼파라미터 후보 범위가 넓거나 연속적인 경우, 모든 경우를 탐색하기 어려울 때 효율적으로 **탐색 시간과 자원**을 절약할 수 있다.

---

**주요 특징**

| 항목 | 설명 |
|------|------|
| **탐색 방식** | 지정한 횟수(`n_iter`)만큼 무작위 샘플링 |
| **평가 방법** | 교차검증(Cross Validation)으로 성능 평가 |
| **장점** | 연산 비용 절감, 고차원 탐색에 유리 |
| **단점** | 완전 탐색이 아니므로 최적값을 놓칠 가능성 존재 |


In [ ]:
#defining a set of values as a dictionary for hyperparameters

param_grid = {
    "n_estimators":[100,200,300,400],
    "max_depth":[1,3,5,7],
    "reg_lambda":[.01,.1,.5]
}

#declaring RandomizedSearchCV model

model = model_selection.RandomizedSearchCV(
    estimator = clf,
    param_distributions = param_grid,
    scoring = 'accuracy',
    verbose = 10,
    n_jobs = 1,
    cv = 5,
    n_iter=10
)

#fitting values to the RandomizedSearchCV model

model.fit(X,y)

#printing the best possible values to enhance accuracy

print(model.best_params_)
print(model.best_estimator_)
#printing the best score
print(model.best_score_)

- 분포를 활용한 RandomSerachCV

In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import model_selection, metrics
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from scipy.stats import randint, uniform, loguniform  # 분포

data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

param_dist = {
    "n_estimators": randint(300, 1500),        # 트리 개수
    "max_depth": randint(2, 8),                # 트리 깊이
    "min_child_weight": randint(1, 10),        # 리프 최소 가중치합
    "subsample": uniform(0.6, 0.4),            # 표본 샘플 비율 [0.6, 1.0)
    "colsample_bytree": uniform(0.6, 0.4),     # 특성 샘플 비율 [0.6, 1.0)
    "gamma": loguniform(1e-3, 1.0),            # 분할 최소 손실 감소
    "reg_lambda": loguniform(1e-2, 10),        # L2
    "reg_alpha": loguniform(1e-3, 1),          # L1
    "learning_rate": loguniform(5e-2, 2e-1),   # 0.05~0.2 근방
}

clf = xgb.XGBClassifier(
    tree_method="hist",
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1,
)

rs = model_selection.RandomizedSearchCV(
    estimator=clf,
    param_distributions=param_dist,
    n_iter=40,                 # 예: 40회 시도 (시간에 맞게 조절)
    scoring="accuracy",        # 요청과 동일하게 accuracy 사용
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42,
)


rs.fit(X_train, y_train)

print("Best Params:", rs.best_params_)
print("CV Best Score (accuracy):", round(rs.best_score_, 4))
best_model = rs.best_estimator_


y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]
print("Test Accuracy:", round(metrics.accuracy_score(y_test, y_pred), 4))
print("Test ROC-AUC :", round(metrics.roc_auc_score(y_test, y_proba), 4))

imp = best_model.feature_importances_
order = np.argsort(imp)
plt.figure(figsize=(7, 8))
plt.barh(np.array(X.columns)[order], imp[order])
plt.title("XGBoost Feature Importance")
plt.tight_layout(); plt.show()

# 3. Bayesian Optimization

---
  
입력값 $ x $에 대한 미지의 목적 함수 $~f(x)~$를 탐색하여 그 함숫값 $~f(x)~$를 **최대화(또는 최소화)하는 최적해**를 찾는 방법이다.

이 방법은 목적 함수를 직접 계산하기 어렵거나 시간이 많이 드는 경우 (예: 딥러닝 모델의 하이퍼파라미터 탐색)에 매우 효과적이다.  

목적 함수를 **black-box function**이라 부르며, 이를 근사하기 위해 확률적 모델(Surrogate Model)을 사용한다.

---

**1) Surrogate Model (대체 모델)**

실제 목적함수 $f(x) $를 직접 계산하지 않고, 현재까지의 관측 $ D_t=\{(x_1,f(x_1)),\dots,(x_t,f(x_t))\} $로 **확률적 근사**를 만든다.  


이 근사 모델이 **각 후보 $x$에서의 예측 평균 $\mu(x)$와 불확실성 $\sigma(x)$를 제공**.

- $f(x)$ 평가가 고비용일 때, 싸고 빠른 근사로 "어디를 다음에 평가할지"결정하기 위함.

- 불확실성 정보를 함께 제공해 탐색(Exploration)을 정량화.

---

| 모델 | 핵심 아이디어 | 장점 | 단점/주의 |
|---|---|---|---|
| **Gaussian Process (GP)** | 모든 점의 $f(x)$가 다변량 정규. 커널 $k(x,x')$로 상관 구조 모델링 | $\mu,\sigma$를 폐형식으로 제공. 불확실성 정량화 우수 | 고차원·대규모 데이터에서 비효율, 커널·하이퍼파라미터 민감 |
| **TPE (Tree-structured Parzen Estimator)** | “좋은 성능”과 “나쁜 성능” 영역의 파라미터 분포를 각각 KDE로 추정 | 범주·연속 혼합, 고차원에도 실용적 | 이론적 해석은 GP보다 덜 투명 |
        "폐형식(closed form)"은 수학·통계에서 명시적인 계산식으로 표현 가능한 형태. 어떤 값을 구하기 위해 반복적 수치계산이나 근사법이 필요 없는 형태를 말함.
        가우시안 프로세스(GP)는 예측값의 평균과 불확실성 을미리 정해진 수식으로 직접 계산할 수 있어서“폐형식으로 계산된다”고 함.

---
### **Gaussian Process (GP)**

입력과 출력 사이의 관계를 커널 함수 $k(x, x')$ 로 표현하여  새로운 입력값에 대한 예측 평균($\mu$)과 불확실성(분산 $\sigma^2$)을 계산하는 모델입니다.

---


- 입력 데이터 : $~X = [x_1, x_2, \dots, x_t] $
- 출력 데이터 : $~y = [f(x_1), f(x_2), \dots, f(x_t)]^\top$
- 새 입력점 : $~x_* $

가우시안 프로세스는 모든 함수값이 다변량 정규분포를 따른다고 가정합니다.

$$
\begin{bmatrix}
y \\ f(x_*)
\end{bmatrix}
\sim
\mathcal{N}\left(
0,
\begin{bmatrix}
K(X,X)+\sigma_n^2 I & K(X,x_*)\\
K(x_*,X) & K(x_*,x_*)
\end{bmatrix}
\right)
$$

여기서  
- $K(X,X)$ : 훈련 데이터 간 커널 행렬  
- $K(X,x_*)$ : 훈련 데이터와 새 입력 간의 커널 벡터  
- $\sigma_n^2 I$ : 관측 노이즈(측정 오차)

---

###**커널 행렬 (Kernel Matrix)**

입력 데이터 간의 유사도(similarity)를 수학적으로 표현한 행렬

가우시안 프로세스(GP)에서는 이 행렬이 공분산 행렬(covariance matrix)의 역할을 하며, 각 입력 쌍 $(x_i, x_j)$에 대해 커널 함수 $k(x_i, x_j)$를 계산하여 구성

---

훈련 데이터가 $ X = [x_1, x_2, ..., x_t] $일 때, 커널 행렬 $ K(X, X) $는 다음과 같이 정의됩니다.

$$
K(X,X) =
\begin{bmatrix}
k(x_1,x_1) & k(x_1,x_2) & \dots & k(x_1,x_t) \\
k(x_2,x_1) & k(x_2,x_2) & \dots & k(x_2,x_t) \\
\vdots & \vdots & \ddots & \vdots \\
k(x_t,x_1) & k(x_t,x_2) & \dots & k(x_t,x_t)
\end{bmatrix}
$$



행 $i$, 열 $j$의 원소는 두 데이터 $x_i$와 $x_j$ 사이의 유사도를 나타냅니다.

---


| 커널 이름 | 수식 | 의미 |
|------------|------|------|
| **RBF (Radial Basis Function)** | $$k(x_i,x_j)=\exp\!\left(-\frac{\|x_i-x_j\|^2}{2l^2}\right)$$ | 거리가 가까울수록 높은 유사도 |
| **Linear Kernel** | $$k(x_i,x_j)=x_i^\top x_j$$ | 선형 관계 표현 |
| **Polynomial Kernel** | $$k(x_i,x_j)=(x_i^\top x_j + c)^d$$ | 다항식 형태의 비선형 관계 |
| **Matern Kernel** | $$k(r)=\frac{1}{\Gamma(\nu)2^{\nu-1}}\!\left(\frac{\sqrt{2\nu}r}{l}\right)^\nu K_\nu\!\left(\frac{\sqrt{2\nu}r}{l}\right)$$ | 함수의 부드러움을 조절 가능 |

---



**2) Acquisition Function (획득 함수)**

Surrogate Model의 $\mu(x),\sigma(x)$를 이용해 다음 평가점 $x_{t+1}$을 고르는 기준 함수 $a(x)$. 핵심은 **탐색–활용 균형**을 수식으로 구현하는 것.

$$
x_{t+1}=\arg\max_x a(x \mid D_t)
$$


**대표 획득 함수**

**(a) Expected Improvement (EI)**

현재 최적값 $f^{+}=\max_{i\le t} f(x_i)$ 대비 기대 개선량의 기댓값.

$$
\text{EI}(x)=\mathbb{E}[(f(x)-f^{+})_+]
$$

GP 가정에서
$$
z=\frac{\mu(x)-f^{+}-\xi}{\sigma(x)},\quad
\text{EI}(x)=(\mu-f^{+}-\xi)\Phi(z)+\sigma\phi(z)
$$
- $\Phi,\phi$: 표준정규 CDF, PDF  
- $\xi\ge0$: 탐색 강화 상수. $\xi$가 크면 탐색 성향 증가.

**특징**: 개선의 “크기”와 “확률”을 동시에 고려. 실무 기본값으로 널리 사용.

**(b) Probability of Improvement (PI)**
개선이 일어날 **확률**만 최대로.

$$
\text{PI}(x)=\mathbb{P}(f(x)>f^{+}+\xi)=\Phi\!\left(\frac{\mu(x)-f^{+}-\xi}{\sigma(x)}\right)
$$
**특징**: 단순하고 빠름. 개선 “크기” 무시 → 미세개선에 집착 위험.

**(c) Upper Confidence Bound (UCB)**
평균과 불확실성의 가중 합.
$$
\text{UCB}(x)=\mu(x)+\kappa\,\sigma(x)
$$
- $\kappa>0$: 탐색 강도. 클수록 미탐색 영역 선호.  
**특징**: 구현 간단, 해석 직관적. \(\kappa\) 튜닝이 성능 좌우.


- **EI**: 안정적 기본 선택. 개선 크기까지 고려.  
- **UCB**: 제어 가능한 탐색–활용 스위치가 필요할 때.  
- **PI**: 계산 단순·고속이 중요할 때나 사전 탐색 단계.

<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fb33tsP%2FbtraMpvxJG0%2FSn7uQK7k910IQ7cP3ZM9vk%2Fimg.png'>

위 그림은 **Bayesian Optimization**의 전체 탐색 과정을 시각화한 것입니다.   상단 그래프는 목적함수 $f(x)$와 그에 대한 예측 모델(Surrogate Model)을 보여주며, 하단 그래프는 획득함수(Acquisition Function, EI)를 통해 다음 실험 위치를 결정하는 과정을 나타냄.

---
상단 그래프 (Surrogate Model)

| 요소 | 설명 |
|------|------|
| **파란색 실선** | 우리가 찾고자 하는 실제 **목적함수 f(x)** |
| **빨간색 점** | 현재까지 관측된 입력값–출력값 데이터 포인트 |
| **검정색 점선** | 관측 데이터를 기반으로 Surrogate Model이 예측한 **평균 함수(Estimated Function)** |
| **파란 음영 영역** | Surrogate Model이 예측한 함수의 **불확실성(Variance, Confidence Interval)** |
| **넓은 음영** | 아직 탐색이 부족하여 불확실성이 큰 구간 |
| **좁은 음영** | 이미 많이 탐색되어 불확실성이 작은 구간 |

즉, Surrogate Model은 실제 함수를 근사하며, 탐색이 진행될수록 예측 곡선(검정 점선)이 실제 함수(파란 실선)에 가까워짐.

---
(Acquisition Function)

| 요소 | 설명 |
|------|------|
| **보라색 곡선 (EI(x))** | Expected Improvement: 개선 가능성이 큰 구간을 나타내는 **획득 함수** |
| **노란 별 표시** | EI(x)가 가장 큰 지점 → **다음 입력값 후보 (Next Candidate)** |

EI(x)가 높은 구간은 "현재까지의 정보로 볼 때 성능이 더 좋아질 가능성이 큰" 구간입니다.  Bayesian Optimization은 이 구간의 입력값을 **다음 실험점으로 선택**



- 직관적인 이해

In [ ]:
# - 목표: f(x)를 최대화할 x 찾기
# - Surrogate: Gaussian Process
# - Acquisition: Expected Improvement(EI)
# - 의존성: scikit-learn, numpy, matplotlib

import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel

# 1) 목적함수 정의. 실제로는 모델 성능(정확도 등)이 들어간다.
def f(x):
    x = np.asarray(x)
    return (np.sin(12*x) + 0.6*np.cos(5*x) + 0.1*np.sin(25*x)) * (0.5 + 4*x)

# 2) 탐색 구간 설정. 학습률 느낌으로 [0.01, 0.09]
bounds = (0.01, 0.09)

# 3) 초기 관측 n개 샘플
rng = np.random.RandomState(42)
n_init = 4
X = rng.uniform(bounds[0], bounds[1], size=(n_init, 1))
y = f(X).ravel()

# 4) EI 함수 구현
from scipy.stats import norm

def expected_improvement(Xcand, model, y_best, xi=1e-3):
    # Xcand: shape (m, 1)
    mu, sigma = model.predict(Xcand, return_std=True)
    sigma = np.maximum(sigma, 1e-9)  # 수치 안정화
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    ei[sigma < 1e-9] = 0.0
    return ei

# 5) BO 루프
n_iter = 10
for t in range(n_iter):
    # GP 적합
    kernel = ConstantKernel(1.0, (1e-3, 1e3)) * RBF(length_scale=0.02, length_scale_bounds=(1e-3, 1.0)) \
             + WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-8, 1e-1))
    gp = GaussianProcessRegressor(kernel=kernel, alpha=0.0, normalize_y=True, random_state=0)
    gp.fit(X, y)

    # 후보 격자에서 EI 최대점 선택
    X_grid = np.linspace(bounds[0], bounds[1], 600).reshape(-1, 1)
    ei = expected_improvement(X_grid, gp, y_best=y.max(), xi=1e-3)
    x_next = X_grid[np.argmax(ei)].reshape(1, 1)

    # 새 점 평가 후 데이터 갱신
    y_next = f(x_next).ravel()
    X = np.vstack([X, x_next])
    y = np.hstack([y, y_next])

# 6) 최종 시각화
gp.fit(X, y)
X_plot = np.linspace(bounds[0], bounds[1], 600).reshape(-1, 1)
mu, std = gp.predict(X_plot, return_std=True)

fig = plt.figure(figsize=(10, 6))

# 상단: Surrogate 예측과 불확실성
ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
ax1.plot(X_plot, f(X_plot), label="Objective f(x)", linewidth=2)
ax1.plot(X_plot, mu, "k--", label="GP Prediction", linewidth=2)
ax1.fill_between(X_plot.ravel(), mu-2*std, mu+2*std, alpha=0.25, label="Uncertainty (±2σ)")
ax1.scatter(X[:-1], y[:-1], color="crimson", edgecolor="k", zorder=5, label="Observations", s=50)
ax1.scatter(X[-1], y[-1], color="gold", edgecolor="k", zorder=6, label="Last eval", s=70)
ax1.set_ylabel("f(x)")
ax1.set_xlim(bounds)
ax1.legend(loc="upper right", frameon=False)
ax1.set_title("Bayesian Optimization with GP + Expected Improvement")

# 하단: EI와 다음 후보
ei = expected_improvement(X_plot, gp, y_best=y.max(), xi=1e-3)
x_star = X_plot[np.argmax(ei)]
ax2 = plt.subplot2grid((3, 1), (2, 0))
ax2.plot(X_plot, ei, label="EI(x)")
ax2.scatter([x_star], [ei.max()], color="gold", edgecolor="k", zorder=6, label="Next candidate", s=70)
ax2.set_xlim(bounds)
ax2.set_xlabel("x")
ax2.set_ylabel("EI(x)")
ax2.legend(loc="upper right", frameon=False)

plt.tight_layout()
plt.show()

# 7) 결과 출력
x_opt_idx = np.argmax(y)
print("관측 중 최고값 x* =", float(X[x_opt_idx]), " f(x*) =", float(y[x_opt_idx]))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
#확률적 회귀 모델, 데이터 포인트 사이의 관계를 확률분포로 모델링 하는 것.
# 특징 : 비선형 관계를 학습할 수 있음. 샘플 개수가 적어도 신뢰성 있는 예측할수 있다.
# 특징 : 불확실성 정량적으로 평가 가능. / 커널 : 데이터 포인트 사이의 유사도를 측정하는 함수.
# 가우시안 프로세스(GP)에서 커널은 "이 점과 저 점이 얼마나 비슷한가?"
#RBF(Radial Basis Function,aka Gaussian kernel)
# 가장 부드러운 커널. 모든 점이 서로 영향을 미칠 가능성이 있음.
# Matern 커널 : RBF보다는 매끄럽지는 않음. 함수의 거칠기(roughtness)조절할 수 있음.
# -> RBF : 매우 부드러운 변화를 가짐. / Matern : 덜 부드럽고 급격한 변화도 허용.
from sklearn.gaussian_process.kernels import Matern, RBF

# 목적 함수 (최적화할 함수)
# 주어진 x 값에 대해 비선형 함수의 값을 반환
# 이 함수는 최적화를 수행할 대상이며, 샘플링을 통해 최적값을 찾아간다.
def objective_function(x):
    return np.sin(3*x) + x**2 - 0.7*x

# 획득 함수 (Expected Improvement, EI)
# 현재까지의 샘플링 데이터를 기반으로 획득 함수 값을 계산하여, 다음 샘플링할 최적의 지점을 결정한다.
def expected_improvement(X, X_sample, Y_sample, gpr, xi=0.01):
    mu, sigma = gpr.predict(X, return_std=True)
    sigma = sigma.reshape(-1, 1)  # 2D 형태 유지
    mu_sample = gpr.predict(X_sample)
    mu_sample_opt = np.max(mu_sample)

    # np.errstate를 사용하여 경고 메시지를 제어
    # divide='warn': 0으로 나누는 연산이 발생할 경우 경고를 발생시키고 오류를 방지
    with np.errstate(divide='warn'):
        improvement = mu - mu_sample_opt - xi
        Z = improvement / sigma  # sigma가 0일 경우 문제가 발생할 수 있음
        ei = improvement * norm.cdf(Z) + sigma * norm.pdf(Z)

        ei = ei.flatten()  # 1D 배열로 변환
        sigma = sigma.flatten()  # 1D 배열로 변환

        if ei.shape == sigma.shape:
            ei[sigma == 0.0] = 0.0  # 0인 값 처리

    return ei

# 베이지안 최적화 실행 함수
# 초기 샘플링 데이터를 기반으로 가우시안 프로세스를 학습하며, 최적의 지점을 찾아간다.
def bayesian_optimization(n_iterations=10, n_initial=3, kernel_type='Matern'):
    np.random.seed(42)

    # 검색 범위
    X = np.linspace(-1, 2, 100).reshape(-1, 1)

    # 초기 샘플링 데이터
    X_sample = np.random.uniform(-1, 2, size=(n_initial, 1))
    Y_sample = objective_function(X_sample)

    # 커널 선택 (Matern 또는 RBF)
    if kernel_type == 'Matern':
        kernel = Matern(length_scale=0.5, nu=2.5)
    elif kernel_type == 'RBF':
        kernel = RBF(length_scale=0.5)
    else:
        raise ValueError("Invalid kernel type. Choose 'Matern' or 'RBF'")

    # 가우시안 프로세스 모델 설정
    gpr = GaussianProcessRegressor(kernel=kernel, alpha=1e-6)

    for i in range(n_iterations):
        # 모델 학습
        gpr.fit(X_sample, Y_sample)

        # 획득 함수 계산 (Expected Improvement 기반)
        ei = expected_improvement(X, X_sample, Y_sample, gpr)
        X_next = X[np.argmax(ei)]

        # 새로운 데이터 추가(Acquisition Function)
        Y_next = objective_function(X_next.reshape(1, -1))
        X_sample = np.vstack((X_sample, X_next.reshape(-1, 1)))
        Y_sample = np.vstack((Y_sample, Y_next.reshape(-1, 1)))

        # 시각화 (최적화 진행 과정 확인)
        plt.figure(figsize=(8, 5))
        mu, sigma = gpr.predict(X, return_std=True)
        plt.plot(X, objective_function(X), 'r--', label='True Function')
        plt.plot(X, mu, 'b-', label='GP Mean')
        plt.fill_between(X.ravel(), mu - 1.96 * sigma, mu + 1.96 * sigma, alpha=0.2, color='blue')
        plt.scatter(X_sample, Y_sample, c='black', label='Samples')
        plt.axvline(X_next, color='green', linestyle='--', label='Next Sample')
        plt.title(f'Iteration {i+1} (Kernel: {kernel_type})')
        plt.legend()
        plt.show()

# 실행 예시 (Matern과 RBF 커널 비교)
bayesian_optimization(kernel_type='Matern')  # Matern 커널 사용
bayesian_optimization(kernel_type='RBF')  # RBF 커널 사용

- xgboost 하이퍼 파라미터 최적화

In [ ]:
#import packages
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris

#Loading iris dataset from sklearn
iris = load_iris()

#independent feautres
X = iris.data

# target features
y = iris.target

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, stratify=y)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
import xgboost as xgb

# MAPE Metric
def mean_absolute_percentage_error(y_test, y_pred):
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

# 탐색 대상 함수 (XGBRegressor)
def XGB_cv(max_depth,learning_rate, n_estimators, gamma
            ,min_child_weight, subsample
            ,colsample_bytree, silent=True, nthread=-1):

    # 모델 정의
    model = xgb.XGBRegressor(max_depth=int(max_depth),
                            learning_rate=learning_rate,
                            n_estimators=int(n_estimators),
                            gamma=gamma,
                            min_child_weight=min_child_weight,
                            subsample=subsample,
                            colsample_bytree=colsample_bytree,
                            nthread=nthread
                            )
    # 모델 훈련
    model.fit(X_train, y_train)

    # 예측값 출력
    y_pred= model.predict(X_test)

    # 각종 metric 계산
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    # 오차 최적화로 사용할 metric 반환
    return r2

In [ ]:
!pip install bayesian-optimization

In [ ]:
#  bayesian-optimization 라이브러리의 BayesianOptimization 클래스 import
from bayes_opt import BayesianOptimization
import numpy as np

# 실험해보고자하는 hyperparameter 집합
pbounds = {'max_depth': (3, 7),
            'learning_rate': (0.01, 0.2),
            'n_estimators': (5000, 10000),
            'gamma': (0, 100),
            'min_child_weight': (0, 3),
            'subsample': (0.5, 1),
            'colsample_bytree' :(0.2, 1)
            }

# Bayesian optimization 객체 생성
# f : 탐색 대상 함수, pbounds : hyperparameter 집합
# verbose = 2 항상 출력, verbose = 1 최댓값일 때 출력, verbose = 0 출력 안함
# random_state : Bayesian Optimization 상의 랜덤성이 존재하는 부분을 통제
bo=BayesianOptimization(f=XGB_cv, pbounds=pbounds, verbose=2, random_state=1 )

# 메소드를 이용해 최대화 과정 수행
# init_points :  초기 Random Search 갯수
# n_iter : 반복 횟수 (몇개의 입력값-함숫값 점들을 확인할지! 많을 수록 정확한 값을 얻을 수 있다.)
# acq : Acquisition Function들 중 Expected Improvement(EI) 를 사용 -> 사라짐(2024.04.12)
# xi : exploration 강도 (기본값은 0.0) -> 사라짐(2024.04.12)
bo.maximize(init_points=2, n_iter=10)

# ‘iter’는 반복 회차, ‘target’은 목적 함수의 값, 나머지는 입력값을 나타냅니다.
# 현재 회차 이전까지 조사된 함숫값들과 비교하여, 현재 회차에 최댓값이 얻어진 경우,
# bayesian-optimization 라이브러리는 이를 자동으로 다른 색 글자로 표시하는 것을 확인할 수 있습니다

# 찾은 파라미터 값 확인
print(bo.max)

# 4.Hyperopt

- HyperOpt는 자동화된 하이퍼파라미터 튜닝 프레임워크로서, fmin()이라는 함수 안에는 3가지의 파라미터가 있다:

    - Objective Function: 최소화할 손실 함수
    - Domain Space: 탐색 범위. 베이지안 최적화에서는 이 범위가 각  하이퍼파라미터에 대해 통계 분포를 만들어낸다.
    - Optimization Algorithm : 최적의 조합을 찾기 위한 알고리즘

참고 : https://velog.io/@emseoyk/%ED%95%98%EC%9D%B4%ED%8D%BC%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0-%ED%8A%9C%EB%8B%9D

In [ ]:
!pip install scikit-optimize

In [ ]:
#importing packages

from hyperopt import hp,fmin, tpe, Trials

from hyperopt.pyll.base import scope

from functools import partial

from skopt import space

from skopt import gp_minimize

#defining a method that will perfrom a 5 split cross validation over

#dataset and and will produce the optimum value of the accuracy

def optimize(params, x,y):

    clf = XGBClassifier(**params)

    kf = model_selection.StratifiedKFold(n_splits=5)

    accuracies = []

    for idx in kf.split(X=x,y=y):

        train_idx,test_idx = idx[0],idx[1]

        xtrain = x[train_idx]

        ytrain = y[train_idx]

        xtest = x[test_idx]

        ytest = y[test_idx]

        clf.fit(xtrain,ytrain)

        preds =  clf.predict(xtest)

        fold_acc = metrics.accuracy_score(ytest,preds)

        accuracies.append(fold_acc)

    return -1.0 * np.mean(accuracies)

#defining a set of values as hp for hyperparameters

param_space = {

    "max_depth" : scope.int(hp.quniform("max_depth",3,20, 1)) ,

    "min_child_weight" : scope.int(hp.quniform("min_child_weight",1,8, 1)),

    "n_estimators": scope.int(hp.quniform("n_estimators",100,1500,1)),

    'learning_rate': hp.uniform("learning_rate",0.01,1),

    'reg_lambda': hp.uniform("reg_lambda",0.01,1),

    'gamma': hp.uniform("gamma",0.01,1),

    'subsample': hp.uniform("subsample",0.01,1)

    }

#defiing optimization_fuction as partial and calling optimize within it

optimization_fuction = partial(optimize,x = X, y = y)

trials = Trials()

#Getting the optimum values for hyperparameters

result = fmin(

    fn = optimization_fuction,

    space = param_space,

    algo = tpe.suggest,

    max_evals = 15,

    trials = trials

)

#Printing the best hyperparemeter set

print(result)

# 5.Optuna

- Optuna는 ML 알고리즘의 하이퍼파라미터 튜닝을 자동화해주는 오픈소스 툴입니다. 유사한 툴로 Hyperopt가 있지만 사용성과 문서화, 시각화 제공 여부 등에서 Optuna의 손을 들어주는 경우가 많음.

- 하이퍼파라미터 튜닝에 쓰고 있는 최신 Automl 기법입니다.
- 빠르게 튜닝이 가능하다는 장점이 있음.
- 하이퍼파라미터 튜닝 방식을 지정할수 있다. -> 직관적인 api인 튜닝된 lightgbm도 제공해줍니다.

- 다른 라이브러리들에 비해 직관적인 장점이 있어 코딩하기 용이합니다.


- 거의 모든 ML/DL 프레임워크에서 사용 가능한 넓은 범용성을 가지고 있다.
간단하고 빠르다.
- 최신 동향의 다양한 최적화 알고리즘을 갖추고 있다.
- 병렬 처리가 가능하다.
- 간단한 메소드로 시각화가 가능하다.

- Optuna를 이해하기 위해서는 다음의 용어에 익숙해져야 한다.

    - Study: 목적 함수에 기반한 최적화
    - Trial: 목적함수 시행
- 쉽게 말해 study는 최적화를 하는 과정이고, trial은 다양한 조합으로 목적함수를 시행하는 횟수를 뜻한다.
- Study의 목적은 여러 번의 trial을 거쳐 최적의 하이퍼파라미터 조합을 찾는 것이라고 할 수 있겠다.

- 직관적인 코드(단변수 함수)

In [ ]:
!pip install optuna

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import optuna

# 목적 함수 (최적화할 함수)
# 이 함수는 우리가 최소화하려는 함수로, x에 대해 비선형적인 패턴을 가짐
def objective_function(x):
    return np.sin(3*x) + x**2 - 0.7*x

# Optuna를 활용한 최적화 예시
def optuna_optimization():
    # Optuna의 최적화 대상이 될 목적 함수 정의
    def objective(trial):
        # x 값을 -1에서 2 사이의 연속적인 실수 값으로 탐색하도록 설정
        x = trial.suggest_float("x", -1, 2)
        return objective_function(x)

    # Optuna의 Study 객체 생성 (최소화 문제로 설정)
    study = optuna.create_study(direction="minimize")

    # 최적화를 수행하며, 총 20번의 탐색을 실행
    study.optimize(objective, n_trials=20)

    # 최적해 출력 (최적의 x 값과 해당하는 목적 함수 값)
    print("Best value:", study.best_value)  # 최소화된 함수 값 출력
    print("Best parameter:", study.best_params)  # 최적의 x 값 출력

    # 최적화 진행 과정 시각화 (목적 함수 값이 어떻게 줄어드는지 확인)
    fig = optuna.visualization.matplotlib.plot_optimization_history(study)
    plt.title("Optimization History")
    plt.show()

    # 탐색된 파라미터의 중요도 분석 (어떤 변수가 최적화에 가장 중요한 영향을 미치는지 확인)
    fig = optuna.visualization.matplotlib.plot_param_importances(study)
    plt.title("Parameter Importance")
    plt.show()

# Optuna 최적화 실행 (함수 호출 시 최적화를 시작)
optuna_optimization()

- 직관적인 이해(다변수)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import optuna

# 목적 함수 (최적화할 함수)
# 이 함수는 우리가 최소화하려는 함수로, x에 대해 비선형적인 패턴을 가짐
def objective_function(x, y):
    return np.sin(3*x) + y**2 - 0.7*x*y

# Optuna를 활용한 다변수 최적화 예시
def optuna_optimization():
    # Optuna의 최적화 대상이 될 목적 함수 정의
    def objective(trial):
        # x 값을 -1에서 2 사이의 연속적인 실수 값으로 탐색하도록 설정
        x = trial.suggest_float("x", -1, 2)
        y = trial.suggest_float("y", -2, 2)
        return objective_function(x, y)

    # Optuna의 Study 객체 생성 (최소화 문제로 설정)
    study = optuna.create_study(direction="minimize")

    # 최적화를 수행하며, 총 50번의 탐색을 실행 (다변수 탐색 강화)
    study.optimize(objective, n_trials=50)

    # 최적해 출력 (최적의 x, y 값과 해당하는 목적 함수 값)
    print("Best value:", study.best_value)  # 최소화된 함수 값 출력
    print("Best parameters:", study.best_params)  # 최적의 x, y 값 출력

    # 최적화 진행 과정 시각화 (목적 함수 값이 어떻게 줄어드는지 확인)
    fig = optuna.visualization.matplotlib.plot_optimization_history(study)
    plt.title("Optimization History")
    plt.show()

    # 탐색된 파라미터의 중요도 분석 (어떤 변수가 최적화에 가장 중요한 영향을 미치는지 확인)
    fig = optuna.visualization.matplotlib.plot_param_importances(study)
    plt.title("Parameter Importance")
    plt.show()

    # x와 y 값의 탐색 과정 시각화 (탐색 경로 분석)
    fig = optuna.visualization.matplotlib.plot_contour(study, params=["x", "y"])
    plt.title("Parameter Contour")
    plt.show()

# Optuna 최적화 실행 (다변수 최적화를 수행)
optuna_optimization()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import optuna

# 목적 함수 (최적화할 함수)
# 이 함수는 우리가 최소화하려는 함수로, x에 대해 비선형적인 패턴을 가짐
def objective_function(x, y):
    return np.sin(3*x) + y**2 - 0.7*x*y

# Optuna를 활용한 다변수 최적화 예시 (고급 기능 추가)
def optuna_optimization():
    # Optuna의 최적화 대상이 될 목적 함수 정의
    def objective(trial):
        # x 값을 -1에서 2 사이의 연속적인 실수 값으로 탐색하도록 설정
        x = trial.suggest_float("x", -1, 2)
        y = trial.suggest_float("y", -2, 2)
        return objective_function(x, y)

    # Optuna의 Study 객체 생성 (최소화 문제로 설정)
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
    #TPE (Tree-structured Parzen Estimator) 기반 샘플링 방법, 기존 샘플 데이터로부터 확률 모델을 학습하여, 더 좋은 후보값을 예측
    #기존의 무작위 샘플링보다 빠르고 효율적인 탐색이 가능 / 특히, 하이퍼파라미터 튜닝에서 많이 사용됨

    # 최적화를 수행하며, 총 100번의 탐색을 실행 (탐색 성능 향상)
    study.optimize(objective, n_trials=100)

    # 최적해 출력 (최적의 x, y 값과 해당하는 목적 함수 값)
    print("Best value:", study.best_value)  # 최소화된 함수 값 출력
    print("Best parameters:", study.best_params)  # 최적의 x, y 값 출력

    # 최적화 진행 과정 시각화 (목적 함수 값이 어떻게 줄어드는지 확인)
    fig = optuna.visualization.matplotlib.plot_optimization_history(study)
    plt.title("Optimization History")
    plt.show()

    # 탐색된 파라미터의 중요도 분석 (어떤 변수가 최적화에 가장 중요한 영향을 미치는지 확인)
    fig = optuna.visualization.matplotlib.plot_param_importances(study)
    plt.title("Parameter Importance")
    plt.show()

    # x와 y 값의 탐색 과정 시각화 (탐색 경로 분석)
    fig = optuna.visualization.matplotlib.plot_contour(study, params=["x", "y"])
    plt.title("Parameter Contour")
    plt.show()

    # 하이퍼파라미터의 조합을 3D 산점도로 시각화
    trials = np.array([[t.params["x"], t.params["y"], t.value] for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(trials[:, 0], trials[:, 1], trials[:, 2], c=trials[:, 2], cmap='viridis', marker='o')
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("Objective Value")
    plt.title("Optimization Search Space")
    plt.show()

# Optuna 최적화 실행 (다변수 최적화를 수행)
optuna_optimization()

- randomsampler VS TPEsampler

In [ ]:
import optuna
import numpy as np

# 최적화할 목적 함수 정의
def objective(trial):
    x = trial.suggest_float("x", -5, 5)
    y = trial.suggest_float("y", -5, 5)
    return (x - 2) ** 2 + (y + 3) ** 2  # (2, -3)에서 최솟값을 가짐

# Random Sampler (무작위 탐색)
study_random = optuna.create_study(direction="minimize", sampler=optuna.samplers.RandomSampler())
study_random.optimize(objective, n_trials=50)
print("Random Sampler Best Value:", study_random.best_value, "Best Params:", study_random.best_params)

# TPE Sampler (TPE 기반 탐색)
study_tpe = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
study_tpe.optimize(objective, n_trials=50)
print("TPE Sampler Best Value:", study_tpe.best_value, "Best Params:", study_tpe.best_params)


- 고급버전(3D)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import optuna
from optuna.trial import TrialState

# 목적 함수 (최적화할 함수)
# 이 함수는 우리가 최소화하려는 함수로, x에 대해 비선형적인 패턴을 가짐
def objective_function(x, y):
    return np.sin(3*x) + y**2 - 0.7*x*y

# Optuna를 활용한 다변수 최적화 예시 (고급 기능 추가)
def optuna_optimization():
    # Optuna의 최적화 대상이 될 목적 함수 정의
    def objective(trial):
        # x 값을 -1에서 2 사이의 연속적인 실수 값으로 탐색하도록 설정
        x = trial.suggest_float("x", -1, 2)
        y = trial.suggest_float("y", -2, 2)
        return objective_function(x, y)

    # Optuna의 Study 객체 생성 (최소화 문제로 설정)
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())

    # 최적화를 수행하며, 총 100번의 탐색을 실행 (탐색 성능 향상)
    study.optimize(objective, n_trials=100)

    # 최적해 출력 (최적의 x, y 값과 해당하는 목적 함수 값)
    print("Best value:", study.best_value)  # 최소화된 함수 값 출력
    print("Best parameters:", study.best_params)  # 최적의 x, y 값 출력

    # 최적화 진행 과정 시각화 (목적 함수 값이 어떻게 줄어드는지 확인)
    fig = optuna.visualization.matplotlib.plot_optimization_history(study)
    plt.title("Optimization History")
    plt.show()

    # 탐색된 파라미터의 중요도 분석 (어떤 변수가 최적화에 가장 중요한 영향을 미치는지 확인)
    fig = optuna.visualization.matplotlib.plot_param_importances(study)
    plt.title("Parameter Importance")
    plt.show()

    # x와 y 값의 탐색 과정 시각화 (탐색 경로 분석)
    fig = optuna.visualization.matplotlib.plot_contour(study, params=["x", "y"])
    plt.title("Parameter Contour")
    plt.show()

    # 하이퍼파라미터의 조합을 3D 산점도로 시각화
    trials = np.array([[t.params["x"], t.params["y"], t.value] for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(trials[:, 0], trials[:, 1], trials[:, 2], c=trials[:, 2], cmap='viridis', marker='o')
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("Objective Value")
    plt.title("Optimization Search Space")
    plt.show()

    # 최적화 과정에서 실패한 시도들에 대한 통계 출력
    complete_trials = study.get_trials(states=(TrialState.COMPLETE,))
    failed_trials = study.get_trials(states=(TrialState.FAIL,))
    print(f"Total completed trials: {len(complete_trials)}")
    print(f"Total failed trials: {len(failed_trials)}")

    # 하이퍼파라미터 값 분포 시각화
    fig = optuna.visualization.matplotlib.plot_parallel_coordinate(study)
    plt.title("Parallel Coordinate Plot")
    plt.show()

# Optuna 최적화 실행 (다변수 최적화를 수행)
optuna_optimization()

In [ ]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import optuna
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 데이터 로드
cancer = load_breast_cancer()
X, y = cancer.data, cancer.target

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optuna를 사용한 하이퍼파라미터 최적화
def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 20, 50),  # 트리의 리프 노드 개수
        'max_depth': trial.suggest_int('max_depth', -1, 10),  # 트리의 최대 깊이
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),  # 학습률
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),  # 부스팅 반복 횟수
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),  # 샘플링 비율
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),  # 특성 선택 비율
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.0001, 1.0),  # L1 정규화
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.0001, 1.0)  # L2 정규화
    }
    model = lgb.LGBMClassifier(boosting_type='gbdt', objective='binary', metric='accuracy', **params)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='logloss', callbacks=[lgb.early_stopping(50)])
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

# Optuna 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# 최적의 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", study.best_params)

# 최적 모델 평가
best_params = study.best_params
best_model = lgb.LGBMClassifier(boosting_type='gbdt', objective='binary', metric='accuracy', **best_params)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("최적 모델의 정확도:", accuracy)

- 업그레이드 버전

In [ ]:
pip install optuna-integration

- Optuna Integration (optuna-integration)
    - Optuna와 다양한 머신러닝 프레임워크(XGBoost, LightGBM, PyTorch 등)를 쉽게 연동하는 라이브러리
    - optuna.integration.lightgbm.LightGBMPruningCallback을 통해 LightGBM과 Optuna를 쉽게 연결 가능.

In [ ]:
import optuna
import lightgbm as lgb
from optuna.integration import LightGBMPruningCallback
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

# 데이터 로드
data = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        "num_leaves": trial.suggest_int("num_leaves", 20, 50),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.1),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
    }

    model = lgb.LGBMClassifier(**params)

    # 조기 종료 및 Pruning 설정
    callbacks = [
        LightGBMPruningCallback(trial, "binary_logloss"),  # log loss가 감소하지 않으면 pruning
        lgb.early_stopping(50),  # 50회 동안 개선이 없으면 조기 종료
    ]

    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric="logloss",
        callbacks=callbacks  # early_stopping_rounds 대신 사용
    )

    y_pred_prob = model.predict_proba(X_test)[:, 1]  # 클래스 1에 대한 확률
    return log_loss(y_test, y_pred_prob)  # log loss 값 반환 (작을수록 좋음)

# Optuna 최적화 실행 (`direction="minimize"`로 변경)
study = optuna.create_study(direction="minimize")  # log loss는 낮을수록 좋음
study.optimize(objective, n_trials=50)

# 최적의 하이퍼파라미터 출력
print("Best parameters:", study.best_params)

- load_iris()

In [ ]:
#importing all the required packages
import optuna
from sklearn.datasets import load_iris
import sklearn.linear_model
import sklearn.metrics
from sklearn.model_selection import train_test_split

#Loading iris dataset from sklearn
iris = load_iris()

#independent feautres
X = iris.data

# target features
y = iris.target

In [ ]:
# 필요한 패키지 임포트
import optuna  # 하이퍼파라미터 최적화 라이브러리
import numpy as np  # 수치 계산을 위한 numpy
from sklearn import model_selection, metrics  # 교차 검증과 평가 지표
from xgboost import XGBClassifier  # XGBoost 분류기
from functools import partial  # partial() 함수를 사용해 인자를 고정

#  `optimize` 함수 정의
# - 5-Fold 교차 검증을 수행하여 최적의 하이퍼파라미터를 찾음
# - Optuna를 활용해 자동으로 최적의 값을 탐색
def optimize(trial, x, y):
    """
    Optuna를 이용해 XGBoost 모델의 최적 하이퍼파라미터를 찾는 함수.
    5-Fold Cross Validation을 사용하여 모델을 평가하고, 최적의 파라미터를 탐색함.

    Parameters:
        trial (optuna.trial): Optuna의 트라이얼 객체 (각 실험의 파라미터 조정)
        x (numpy.ndarray): 입력 데이터 (Feature)
        y (numpy.ndarray): 출력 데이터 (Label)

    Returns:
        float: 평균 정확도의 음수 값 (최소화를 위해 -1을 곱함)
    """

    #  Optuna를 이용해 하이퍼파라미터 범위 설정
    # - `suggest_uniform`: 연속된 값에서 랜덤 샘플링 (균등 분포)
    # - `suggest_int`: 정수 값 범위에서 랜덤 샘플링
    reg_lambda = trial.suggest_uniform('reg_lambda', 0.01, 1.0)  # L2 정규화 (0.01 ~ 1.0)
    n_estimators = trial.suggest_int('n_estimators', 100, 1500)  # 트리 개수 (100 ~ 1500)
    max_depth = trial.suggest_int('max_depth', 3, 15)  # 트리 최대 깊이 (3 ~ 15)
    max_features = trial.suggest_uniform('max_features', 0.01, 1.0)  # 선택할 특성 비율 (0.01 ~ 1.0)

    #  XGBoost 모델 생성 (하이퍼파라미터 적용)
    clf = XGBClassifier(
        n_estimators=n_estimators,  # 부스팅 반복 횟수
        reg_lambda=reg_lambda,  # L2 정규화 (규제를 통해 과적합 방지)
        max_depth=max_depth,  # 트리의 최대 깊이 (깊을수록 복잡한 모델)
        max_features=max_features  # 특성 선택 비율 (작을수록 모델이 단순해짐)
    )

    #  5-Fold Stratified K-Fold 교차 검증 설정
    # - 데이터를 5개의 Fold로 나눠, 각 Fold마다 모델을 학습 & 평가
    # - `StratifiedKFold`를 사용해 클래스 비율을 유지하도록 설정 (불균형 데이터 방지)
    kf = model_selection.StratifiedKFold(n_splits=5)

    #  교차 검증을 수행하며 정확도를 저장할 리스트
    accuracies = []

    #  각 Fold에 대해 학습 & 평가 반복
    for idx in kf.split(X=x, y=y):
        train_idx, test_idx = idx[0], idx[1]  # 학습 데이터와 검증 데이터의 인덱스 분할

        #  학습 데이터와 검증 데이터 생성
        xtrain, xtest = x[train_idx], x[test_idx]
        ytrain, ytest = y[train_idx], y[test_idx]

        #  모델 학습
        clf.fit(xtrain, ytrain)

        #  검증 데이터에 대한 예측 수행
        preds = clf.predict(xtest)

        #  Fold별 정확도(Accuracy) 계산
        fold_acc = metrics.accuracy_score(ytest, preds)
        accuracies.append(fold_acc)  # 정확도 리스트에 저장

    #  최종적으로 5-Fold 교차 검증 평균 정확도를 반환
    # - Optuna는 기본적으로 최소화(`direction='minimize'`)를 수행하므로, 음수(-1)를 곱해 반환
    return -1.0 * np.mean(accuracies)

#  `partial()`을 이용해 X와 Y 데이터를 `optimize` 함수에 고정
# - `optimize()` 함수의 인자를 고정하여, `study.optimize()`에서 간편하게 호출 가능
optimization_function = partial(optimize, x=X, y=y)

#  Optuna Study 생성 (하이퍼파라미터 최적화 설정)
# - `direction='minimize'`: 가장 낮은 정확도를 찾는 것이 아니라, `-accuracy` 값을 최소화하여 정확도를 최대화하는 구조
study = optuna.create_study(direction='minimize')

#  Optuna 최적화 실행 (15번의 실험 진행)
study.optimize(optimization_function, n_trials=15)

#  최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", study.best_params)


- mimimize로 찾은 이유 : https://stackoverflow.com/questions/72193393/find-the-value-of-variables-to-maximize-return-of-function-in-python

## Create an objective function

In [ ]:
#  Optuna 개념 정리
# - `Trial`: 하나의 실험(객체)으로, objective 함수가 한 번 실행될 때마다 생성됨.
# - `Study`: 여러 개의 `Trial`을 포함하는 하나의 최적화 세션.
# - `Study`는 특정 목표(objective function)에 따라 최적의 하이퍼파라미터를 찾음.
# - `Trial`은 objective 함수를 실행할 때마다 새로운 하이퍼파라미터 값을 평가함.

#  `objective` 함수 정의
# - Optuna가 하이퍼파라미터를 최적화할 때 호출되는 함수.
# - 여기서는 Lasso Regression 모델의 `alpha` 값을 최적화하는 것이 목표.
def objective(trial):

    #  Optuna를 사용한 하이퍼파라미터 탐색
    # - `suggest_uniform(name, low, high)`: 지정한 범위 내에서 균등 분포를 따르는 값을 샘플링함.
    # - 여기서는 `alpha`를 0.0 ~ 2.0 범위에서 최적의 값을 찾음.
    alpha = trial.suggest_uniform('alpha', 0.0, 2.0)

    #  데이터 로드 및 학습/검증 데이터 분할
    # - `load_iris(return_X_y=True)`: Iris 데이터셋을 가져와 X(특성), y(레이블)로 반환.
    X, y = load_iris(return_X_y=True)

    # - `train_test_split()`: 데이터를 학습(train)과 검증(validation) 세트로 분할.
    # - `random_state=0`: 실험의 재현성을 보장하기 위해 랜덤 시드를 고정.
    X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0)

    #  모델 학습 및 평가
    # - `sklearn.linear_model.Lasso(alpha=alpha)`: Lasso 회귀 모델 생성 (alpha는 정규화 강도를 조절).
    model = sklearn.linear_model.Lasso(alpha=alpha)

    # - `fit(X_train, y_train)`: 학습 데이터로 모델을 학습.
    model.fit(X_train, y_train)

    # - `predict(X_val)`: 검증 데이터로 예측 수행.
    y_pred = model.predict(X_val)

    #  평가 지표: 평균 제곱 오차(MSE)
    # - `mean_squared_error(y_val, y_pred)`: 예측값과 실제값 간의 평균 제곱 오차를 계산.
    error = sklearn.metrics.mean_squared_error(y_val, y_pred)

    #  최적화 목표: MSE를 최소화하는 방향으로 `alpha` 값을 찾음.
    # - Optuna는 반환된 값을 기준으로 최적의 하이퍼파라미터를 탐색함.
    return error

## Create an study for that ML model and optimize it

In [ ]:
import sklearn
# In Optuna, we use the study object to manage optimization.
# Method :func:`~optuna.create_study` returns a study object.
# A study object has useful properties for analyzing the optimization outcome.
study = optuna.create_study(direction='minimize') #Set minimize for minimization and maximize for maximization.
#To start the optimization, we create a study object and pass the objective function to method
study.optimize(objective, n_trials=50)

## Visualized the above hyperparameter optimization study

In [ ]:
#importing all the plot functions
from optuna.visualization import plot_edf
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [ ]:
# Visualize the optimization history. See :func:`~optuna.visualization.plot_optimization_history` for the details.
plot_optimization_history(study)

In [ ]:
# Visualize high-dimensional parameter relationships. See :func:`~optuna.visualization.plot_parallel_coordinate` for the details.
plot_parallel_coordinate(study)

In [ ]:
# Visualize individual hyperparameters as slice plot. See :func:`~optuna.visualization.plot_slice` for the details.
plot_slice(study)

In [ ]:
# Visualize parameter importances. See :func:`~optuna.visualization.plot_param_importances` for the details.
#In this case, we have only one parameter.
plot_param_importances(study)

In [ ]:
# Visualize empirical distribution function. See :func:`~optuna.visualization.plot_edf` for the details.
plot_edf(study)

In [ ]:
import optuna
import sklearn
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

# 1. 최소화/최대화할 목적함수 정의
def objective(trial):
    iris = sklearn.datasets.load_iris()
    x, y = iris.data, iris.target

# 2. trial object로 하이퍼파라미터 값 추천
# 다양한 분류모델을 설정해서 비교할 수 있다.
    classifier_name = trial.suggest_categorical('classifier', ['SVC', 'RandomForest'])
    #분류 모델이 SVC일 때
    if classifier_name == 'SVC':
        svc_c = trial.suggest_loguniform('svc_c', 1e-10, 1e10)
        classifier_obj = sklearn.svm.SVC(C=svc_c, gamma='auto')

    #분류모델이 랜덤포레스트일 때
    else:
        rf_max_depth = int(trial.suggest_loguniform('rf_max_depth', 2, 32))
        classifier_obj = sklearn.ensemble.RandomForestClassifier(max_depth=rf_max_depth, n_estimators=10)

    accuracy = cross_val_score(classifier_obj, x, y, cv = 4).mean()
    return accuracy

# 3. study 오브젝트 생성하고 목적함수 최적화하는 단계
# 여기서는 목적함수를 정확도로 설정했기 때문에 최대화를 목표로 하고 있지만, 손실함수의 경우 direction='minimize'로 설정
study = optuna.create_study(direction='maximize')
# 반복 시행 횟수(trial)는 200번으로
study.optimize(objective, n_trials=200)

In [ ]:
# 시행된 trial 중 최적의 하이퍼파라미터 반환하는 메소드
print(study.best_trial.params)

# 시행된 trial 중 가장 높은 값 반환하는 메소드
optuna_acc = study.best_trial.value
print(optuna_acc)

## Visualization

In [ ]:
optuna.visualization.plot_param_importances(study)

optuna.visualization.plot_optimization_history(study)

#6. TPOT (Tree-based Pipeline Optimization Tool)
- TPOT은 자동 머신러닝(AutoML) 라이브러리 중 하나로, **유전 알고리즘(Genetic Algorithm, GA)**을 활용하여 최적의 머신러닝 파이프라인을 찾아주는 도구

- 여러 개의 모델을 탐색하며, 최적의 Stacking 구조를 찾아줌.
- Python에서 AutoML을 수행하기위한 오픈 소스 라이브러리
- 데이터 변환 및 기계 학습 알고리즘에 인기있는 Scikit-Learn 기계 학습 라이브러리를 사용
- 주어진 데이터 세트에 대한 최고 성능의 모델 파이프 라인을 효율적으로 검색
- 유전 프로그래밍 확률적 글로벌 검색 절차를 사용

<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fb8ctX7%2Fbtrnji5Ub3h%2FKxKKKtTK0TMtC83zUaVnq1%2Fimg.png'>

## 유전 알고리즘

- **유전 알고리즘(Genetic Algorithm, GA)**은 진화론의 개념을 기반으로 최적의 해를 탐색하는 최적화 알고리즘
- 자연 선택(Natural Selection)과 돌연변이(Mutation) 원리를 사용하여, 문제 해결을 위한 최적의 해를 점진적으로 찾아가는 방식

In [ ]:
#예제
import numpy as np

# 목표 함수: -x^2 + 10x + 5 (최대값을 찾기)
def fitness_function(x):
    return -x**2 + 10*x + 5  # 예제 함수 (파라미터 x에 대한 평가)

# 초기 개체군(10개의 무작위 값)
population_size = 10
population = np.random.randint(0, 10, population_size)

# 세대 수 설정
generations = 20

for generation in range(generations):
    # 1. 적합도 평가
    fitness = np.array([fitness_function(ind) for ind in population])

    # 2. 선택 (적합도가 높은 개체 선택)
    selected_parents = population[np.argsort(fitness)][-4:]  # 상위 4개 선택

    # 3. 교차 (선택된 개체들끼리 섞어서 새로운 개체 생성)
    children = (selected_parents[:2] + selected_parents[2:]) // 2  # 단순 평균 교차

    # 4. 돌연변이 (랜덤하게 변화)
    mutation_prob = 0.2
    for i in range(len(children)):
        if np.random.rand() < mutation_prob:
            children[i] += np.random.randint(-1, 2)  # -1 ~ 1 값 추가

    # 5. 새로운 개체군 구성 (부모 + 자식)
    population = np.concatenate((selected_parents, children))

    # 최적해 출력
    best_solution = population[np.argmax(fitness)]
    print(f"Generation {generation+1}: Best solution: x={best_solution}, Fitness={fitness_function(best_solution)}")


- iris dataset으로 tpot를 해보자.

In [ ]:
pip install tpot

In [ ]:
from tpot import TPOTClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

tpot = TPOTClassifier(
    generations=5,
    population_size=20,
    random_state=42,
    n_jobs=1,              # ← 병렬 OFF
    max_time_mins=None     # ← 시간 제한 제거(경고 방지). 쓰면 generations는 빼기
)
tpot.fit(X_train, y_train)
print(tpot.fitted_pipeline_)
print("Accuracy:", accuracy_score(y_test, tpot.predict(X_test)))


import joblib
joblib.dump(tpot.fitted_pipeline_, 'tpot_best_pipeline.pkl')
print("Pipeline saved as tpot_best_pipeline.pkl")

# 이후 불러올 때
# pipeline = joblib.load('tpot_best_pipeline.pkl')
# pipeline.predict(X_test)

## 7. H2O

- h2o는 java 기반의 머신러닝/AI 플랫폼인데요.

- 드라이버가 필요없이 간단한 설정만으로 인공지능을 학습하고 예측할 수 있는 소프트웨어

- 인공지능에 대한 지식이 부족하여도 좋은 모델을 누구나 이용할 수 있는 소프트웨어가 개발되었음.

- H2O는 머신러닝 모델을 대규모 데이터에서 효율적으로 학습할 수 있도록 설계된 오픈소스 AutoML 프레임워크.
- H2O에서는 다양한 머신러닝 알고리즘을 지원하며, 하이퍼파라미터 튜닝(Hyperparameter Optimization, HPO) 기능을 활용하여 최적의 모델을 찾을 수 있어.




In [ ]:
# Java 설치
!sudo apt-get update -qq
!sudo apt-get install -y openjdk-11-jre-headless

# 파이썬 패키지
!pip install -q h2o

# 환경변수 확인(선택)
!java -version
!python -c "import h2o; h2o.init(max_mem_size='2G', nthreads=-1); h2o.shutdown(prompt=False)"

In [ ]:
import h2o
import pandas as pd
from sklearn.datasets import load_iris
from h2o.estimators import H2OXGBoostEstimator   # H2O의 XGBoost 래퍼
from h2o.frame import H2OFrame

# 1) H2O 클러스터 시작
#    - max_mem_size: JVM 메모리 상한
h2o.init(max_mem_size="2G")

# 2) 데이터 준비 → H2OFrame 변환
iris = load_iris(as_frame=True)
df = iris.frame.copy()
# H2O는 공백/특수문자 없는 컬럼명을 권장
df.columns = [c.replace(" (cm)", "").replace(" ", "_") for c in df.columns]
hf: H2OFrame = h2o.H2OFrame(df)

# 다중분류이므로 타깃을 범주형으로 변환
response = "target"
predictors = [c for c in hf.columns if c != response]
hf[response] = hf[response].asfactor()

# 3) 학습/검증/테스트 분할
train, valid, test = hf.split_frame(ratios=[0.7, 0.15], seed=7777)

# 4) H2O XGBoost 모델 정의
#    주요 파라미터는 모두 주석으로 설명
xgb = H2OXGBoostEstimator(
    # ---------- 핵심 학습 설정 ----------
    distribution="multinomial",   # 다중분류 손실. (이진이면 "bernoulli")
    ntrees=500,                   # 트리 개수 상한. 조기종료가 켜져 있으면 실사용 트리는 더 적어짐
    learn_rate=0.1,               # learning_rate. 작을수록 일반화↑, 더 많은 트리 필요
    max_depth=4,                  # 트리 최대 깊이. 과적합 ↔ 표현력 트레이드오프
    min_rows=2,                   # 리프 최소 행 수(min_child_weight에 해당하는 H2O 스타일 파라미터)
    sample_rate=0.8,              # 행 샘플링 비율(subsample). 분산↓, 편향↑
    col_sample_rate=0.8,          # 열 샘플링 비율(colsample_bytree)
    # col_sample_rate_per_tree=0.8,# 트리마다 다른 열 샘플링을 원하면 사용
    # col_sample_rate_per_level=1.0,# 레벨별 열 샘플링(잘 모르면 기본)

    # ---------- 분할/규제 ----------
    min_split_improvement=0.0,    # 분할에 필요한 최소 손실 감소(= XGBoost의 gamma와 유사)
    gamma=0.0,                    # XGBoost의 gamma(일부 빌드에서 min_split_improvement와 함께 사용 가능)
    reg_lambda=1.0,               # L2 규제(가중치 페널티). 과적합 억제 기본축
    reg_alpha=0.0,                # L1 규제(가중치 절대값 페널티). 희소성 유도

    # ---------- 학습 제어 ----------
    stopping_rounds=20,           # 조기종료 라운드. 검증 성능이 향상되지 않으면 학습 중단
    stopping_metric="logloss",    # 조기종료 기준 지표. 다중분류는 logloss/mean_per_class_error 권장
    stopping_tolerance=1e-3,      # 향상 최소 한계
    score_tree_interval=0,        # 0이면 매 트리 후 점수화(검증 계산). 큰 값이면 덜 자주 평가

    # ---------- 엔진/리소스 ----------
    # nthreads=-1,                  # CPU 스레드 수. -1이면 모두 사용
    ## nthreads 인자는 H2O의 XGBoost 버전에서는 지원되지 않는다.
    ##(이는 XGBoost 원본 파라미터이고, H2O는 내부적으로 병렬 처리를 자동 제어함)
    # backend="auto",             # H2O XGBoost 백엔드 선택: "auto"/"cpu"/"gpu"
    # gpu_id=0,                   # GPU 사용 시 장치 ID
    # tree_method="hist",         # 내부 트리 알고리즘. 보통 자동 선택
    # grow_policy="depthwise",    # 트리 성장 정책. "lossguide"는 얕고 넓게 성장

    # ---------- 재현성 ----------
    seed=7777                     # 무작위 시드
)

# 5) 학습
xgb.train(x=predictors, y=response, training_frame=train, validation_frame=valid)

# 6) 검증/테스트 성능
perf_valid = xgb.model_performance(valid)
perf_test  = xgb.model_performance(test)

print("[VALID] logloss:", perf_valid.logloss())
print("[VALID] mean_per_class_error:", perf_valid.mean_per_class_error())
print("[TEST ] logloss:", perf_test.logloss())
print("[TEST ] mean_per_class_error:", perf_test.mean_per_class_error())
print("\n[TEST Confusion Matrix]\n", perf_test.confusion_matrix())

# 7) 변수 중요도
#    - H2O XGBoost는 Gain 기반 varimp 제공
try:
    print("\n[VarImp]\n", xgb.varimp(use_pandas=True))
except Exception as e:
    print("\n[VarImp] 지원 안 함:", e)

# 8) 예측 예시
pred = xgb.predict(test)
print("\n[Predict Head]\n", pred.head())

# 9) (선택) 클러스터 종료
# h2o.shutdown(prompt=False)

### AutoGluon
- Amazon에서 개발한 강력한 AutoML 프레임워크로, 다양한 Stacking 모델을 자동으로 탐색.
- 여러 층의 Stacking을 최적화하며, 내부적으로 Neural Network까지 사용 가능.

이 밖에 AutoML을 할 수 있는 라이브러리들은 다양하게 존재합니다:

- FLAML
- EvalML
- AutoKeras
- Auto-ViML
- AutoGluon
- MLBox

- 참고 : 나는 모델 고민할 시간에 Autogluon을 써(https://dacon.io/codeshare/7764)

[Hyperparam Tuning & AutoML](https://www.kaggle.com/code/sudalairajkumar/hyperparam-tuning-automl)

In [ ]:
!pip install autogluon

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from autogluon.tabular import TabularPredictor

# 1) 데이터 준비
iris = load_iris(as_frame=True)
df = iris.frame.copy()
df.rename(columns={"target": "label"}, inplace=True)  # AutoGluon의 label 이름
# 멀티클래스용 범주형 레이블 권장(가독성 올라감). 숫자 그대로 둬도 됨.
df["label"] = df["label"].map(dict(enumerate(iris.target_names))).astype("category")

train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df["label"]
)

# 2) 학습 설정
# - label: 타깃 컬럼명
# - problem_type="multiclass": 멀티클래스 분류
# - eval_metric="accuracy": 내부 선택 지표
# - presets:
#   "medium_quality" → 속도/성능 균형
#   "best_quality"   → 느리지만 성능 우선(스태킹/배깅 적극 사용)
# - time_limit: 초 단위 전체 탐색 시간 제한
# - num_bag_folds: 배깅 K-폴드 수(>0이면 앙상블)
# - num_stack_levels: 스태킹 레벨(0이면 스태킹 비활성)
save_path = "autogluon_iris"

predictor = TabularPredictor(
    label="label",
    problem_type="multiclass",
    eval_metric="accuracy",
    path=save_path
).fit(
    train_data=train_df,
    presets="medium_quality",
    time_limit=120,        # 2분 제한. 더 주면 성능↑
    num_bag_folds=5,      # 배깅 활성화
    num_stack_levels=1,   # 1단 스태킹
    # hyperparameters=None # 필요 시 모델군/파라미터 직접 지정 가능
)

# 3) 평가
# predictor.evaluate()는 accuracy 외 내부 지표도 출력
perf = predictor.evaluate(test_df)

# 추가로 수동 지표
y_true = test_df["label"]
y_pred = predictor.predict(test_df.drop(columns=["label"]))
print("\nAccuracy:", round(accuracy_score(y_true, y_pred), 4))
print("\nReport:\n", classification_report(y_true, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))

# 4) 리더보드 확인
# - score_val: 검증 점수
# - pred_time_val: 예측 시간
# - fit_time: 학습 시간
lb = predictor.leaderboard(test_df, silent=True)
print("\n[Leaderboard Top 5]\n", lb.head())

# 5) 특성 중요도
# - positive 값이 클수록 정확도에 기여
# - 멀티클래스에서도 전반적 중요도 제공
fi = predictor.feature_importance(test_df)
print("\n[Feature Importance]\n", fi.head())

# 6) 확률 예측(클래스별)
proba = predictor.predict_proba(test_df.drop(columns=["label"]))
print("\n[Predict Proba head]\n", proba.head())

# 7) 모델 저장/재로드
# - fit 시 path에 저장됨. 필요 시 load로 재사용
from autogluon.tabular import TabularPredictor as TPLoad
loaded = TPLoad.load(save_path)
print("\n[Loaded predictor OK] Same acc?:",
      round(accuracy_score(y_true, loaded.predict(test_df.drop(columns=['label']))), 4))


# 8.Pycaret

- AutoML을 하게 해주는 파이썬 라이브러리

- scikit-learning 패키지를 기반으로 하고 있으며 Classification, Regression, Clustering, Anomaly Detection 등 다양한 모델을 지원함.

- 공식문서에 설명이 매우 잘 되어 있고, 몇 줄의 코드로 쉽게 구현이 가능하기 때문에 유용하게 사용할 수 있음.

- Pycaret을 활용하면 여러 모델의 성능 비교 뿐만 아니라 hyperparameter tunning,  여러 모델을 blending한 모델을 만들 수 있습니다.

In [ ]:
!pip install --upgrade pycaret
#설치 후 런타임 다시 시작

####  PyCaret을 사용한 회귀(Regression) 예제

In [ ]:
# PyCaret 회귀 예제 (모델 비교 포함)
import pandas as pd
import matplotlib.pyplot as plt
from pycaret.regression import *

# PyCaret 내장 데이터셋 로드 (Boston Housing 데이터 사용)
from pycaret.datasets import get_data
data = get_data('boston')

# 데이터 분할 (80% 학습, 20% 테스트)
train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)

# PyCaret 환경 설정 (verbose=False로 설정하여 출력 최소화)
regression_setup = setup(data=train_data, target='medv', session_id=42, verbose=False)

# 모든 모델 성능 비교
models_comparison = compare_models(n_select=5)  # 상위 5개 모델 선택

#모델 비교 결과를 DataFrame으로 저장
models_results = pull()  # compare_models() 실행 후 pull()을 사용하여 결과 저장
print(models_results)  # 모델 성능 비교 결과 출력

#최적 모델 선택
best_model = models_comparison[0]  # 가장 좋은 성능의 모델

#최적 모델 학습
final_model = finalize_model(best_model)

# 테스트 데이터 예측
predictions = predict_model(final_model, data=test_data)

#모델 성능 비교 시각화
plt.figure(figsize=(12, 6))
models_results.set_index('Model')['R2'].plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Regression Model Comparison (R^2 Score)')
plt.ylabel('R^2 Score')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


Pycaret는 파이썬 3.12부터는 아직 지원을 하지 않음 (2025-11-03)

### PyCaret을 사용한 분류(Classification) 예제

In [ ]:
# PyCaret 분류 예제 (모델 비교 포함)
import pandas as pd
import matplotlib.pyplot as plt
from pycaret.classification import *

# PyCaret 내장 데이터셋 로드 (타이타닉 데이터)
from pycaret.datasets import get_data
data = get_data('titanic')

# 데이터 전처리 (결측치 제거)
data = data.dropna(subset=['Age', 'Embarked', 'Fare', 'Sex'])

# 범주형 변수 변환 (PyCaret이 자동 변환 가능하지만 명시적으로 처리)
data['Sex'] = data['Sex'].astype(str)
data['Embarked'] = data['Embarked'].astype(str)

# 훈련/테스트 데이터 분할 (80% 학습, 20% 테스트)
train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)

# PyCaret 환경 설정 (verbose=False로 설정하여 출력 최소화)
classification_setup = setup(data=train_data, target='Survived', session_id=42, verbose=False)

# 모델 비교 (상위 5개 모델 선택)
models_comparison = compare_models(n_select=5)

# 모델 비교 결과를 DataFrame으로 저장
models_results = pull()  # compare_models() 실행 후 pull()을 사용하여 결과 저장
print(models_results)  # 모델 성능 비교 결과 출력

# 최적 모델 선택
best_model = models_comparison[0]  # 가장 성능이 좋은 모델

# 최적 모델 학습
final_model = finalize_model(best_model)

# 테스트 데이터 예측
predictions = predict_model(final_model, data=test_data)

# 결과 출력 (실제값과 예측값 비교)
print(predictions[['Survived', 'prediction_label']])

# 모델 성능 비교 시각화 (Accuracy 기준)
plt.figure(figsize=(12, 6))
models_results.set_index('Model')['Accuracy'].plot(kind='bar', color='lightcoral', edgecolor='black')
plt.title('Classification Model Comparison (Accuracy)')
plt.ylabel('Accuracy Score')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


**참고)  [[Public 1st] 고민할 시간이 없어서 [AutoGluon]](https://dacon.io/competitions/official/236075/codeshare/7803?page=1&dtype=recent)**

** AutoGluon : https://auto.gluon.ai/stable/index.html


# 8. FLAML (Fast and Lightweight AutoML)

- FLAML(Fast and Lightweight AutoML)은 Microsoft에서 개발한 가벼운 AutoML 라이브러리로, 빠르고 효율적인 모델 탐색을 목표로 합니다.

### FLAML의 주요 특징
- 빠른 실행 속도
- 다른 AutoML 라이브러리보다 훨씬 빠르게 최적 모델을 찾을 수 있음.
- 가벼운 메모리 사용
- 메모리 효율이 뛰어나 대규모 데이터에도 적합.
- 다양한 머신러닝 알고리즘 지원
- XGBoost, LightGBM, Random Forest, LGBM, LSTM 등 지원.
- 손쉬운 사용법
- 몇 줄의 코드만으로 최적 모델 탐색 가능.
- 하이퍼파라미터 자동 튜닝
- Bayesian Optimization과 비슷한 탐색 방법 사용.

In [ ]:
pip install flaml

- 분류 문제

In [ ]:
import numpy as np
import pandas as pd
from flaml import AutoML
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 데이터 로드 (Iris 데이터셋)
X, y = load_iris(return_X_y=True)

# 데이터 분할 (Train/Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# FLAML AutoML 객체 생성
automl = AutoML()

# AutoML 학습
automl.fit(
    X_train, y_train,
    task="classification",  # 분류 문제
    time_budget=60,  # 최대 실행 시간 (초)
    metric="accuracy",  # 평가 지표
    verbose=0  # 출력 최소화
)

# 최적 모델 확인
print(f"Best model: {automl.best_estimator}")

# 예측
y_pred = automl.predict(X_test)

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"FLAML AutoML Accuracy: {accuracy:.5f}")

- 회귀 문제

In [ ]:
import numpy as np
import pandas as pd
from flaml import AutoML
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 데이터 로드 (Diabetes 데이터셋)
X, y = load_diabetes(return_X_y=True)

# 데이터 분할 (Train/Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# FLAML AutoML 객체 생성
automl = AutoML()

# AutoML 학습
automl.fit(
    X_train, y_train,
    task="regression",  # 회귀 문제
    time_budget=60,  # 최대 실행 시간 (초)
    metric="mse",  # 평가 지표 (Mean Squared Error)
    verbose=0  # 출력 최소화
)

# 최적 모델 확인
print(f"Best model: {automl.best_estimator}")

# 예측
y_pred = automl.predict(X_test)

# MSE 평가
mse = mean_squared_error(y_test, y_pred)
print(f"FLAML AutoML MSE: {mse:.5f}")

#9. LazyPredict

- 여러 머신러닝 모델을 자동으로 학습하고,  **성능($R^2$, Accuracy 등)을 한눈에 비교**할 수 있게 해주는 간단한 라이브러리입니다.  
- 자동 하이퍼파라미터 튜닝은 수행하지 않으며, “기본 설정(default)” 모델을 빠르게 학습합니다.  
- 즉, **Baseline 성능 비교**에 적합합니다.

In [ ]:
!pip install lazypredict

In [ ]:
#분류 모델
from lazypredict.Supervised import LazyClassifier
from sklearn.datasets import load_iris

# 데이터 로드
iris = load_iris(as_frame=True)
X = iris.data
y = iris.target

# 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7777, stratify=y)

# LazyClassifier 생성
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

# 학습 및 평가
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

# 결과 출력
print("LazyPredict Classification Results")
display(models.sort_values(by="Accuracy", ascending=False).head(10))

In [ ]:
top10 = models.sort_values(by="Accuracy", ascending=False).head(10)
plt.figure(figsize=(10,5))
plt.barh(top10.index, top10["Accuracy"], color='skyblue')
plt.xlabel("Accuracy")
plt.title("Top 10 Classification Models by LazyPredict")
plt.gca().invert_yaxis()
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
#회귀
from lazypredict.Supervised import LazyRegressor
from sklearn.datasets import load_diabetes

# 데이터 로드
data = load_diabetes(as_frame=True)
X = data.data
y = data.target

# 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LazyRegressor 생성
reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)

# 학습 및 평가
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

# 결과 출력
print("LazyPredict Regression Results")
display(models.sort_values(by="R-Squared", ascending=False).head(10))

In [ ]:
top10_reg = models.sort_values(by="R-Squared", ascending=False).head(10)
plt.figure(figsize=(10,5))
plt.barh(top10_reg.index, top10_reg["R-Squared"], color='lightcoral')
plt.xlabel("R^2 (Coefficient of Determination)")
plt.title("Top 10 Regression Models by LazyPredict")
plt.gca().invert_yaxis()
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()


참고)

### Numba
 - Python 및 Numpy 코드의 하위 집합을 빠른 기계 코드로 변환하는 오픈소스 Jit 컴파일러.

 - Just In Time 컴파일러를 사용해 파이썬 코드 내에서 일반 코드 및 Numpy를 아주 빠른 속도로 처리 가능한 기능을 제공.

- 공식문서에 설명
    - Numba makes Python code fast.
    - Numba is an open source JIT compiler that translates a subset of Python and NumPy code into fast machine code.

- Numbda 작동원리
    - 데커레이팅된 함수에 대한 파이썬 bytecode를 읽고 이를 함수의 입력 인수 유형에 대한 정보와 결합한다,.
    - 코드를 분석하고 최적화한 후, LLVM compiler library를 사용하여 함수의 machine code 버전을 만들고, 이를 사용자의 CPU 능력에 맞춤.
    - 이 complied된 버전이 앞으로 그 함수를 호출할 때마다 사용된다.

- Numbda 모듈이 모든 파이썬 코드를 최적화 해주는 것은 아니다. 일부 파이썬 코드와 Numpy 대해서만 작동하며 다른 모듈을 이용한 코드를 최적화 시켜주지는 못한다. ex)Numbda는 Pandas를 이해하지 못함. 특정 목적에 따라 충분히 활용할 수 있는 가치가 있는 모듈.

In [ ]:
!pip install numba

In [ ]:
from time import perf_counter
from numba import jit

# 일반적인 loop
def pure_sum(n):
    result = 0
    for i in range(0, n+1):
        result += i
    return result

# Numba 모듈 사용
@jit(nopython=True, cache=True)
def numba_sum(n):
    result = 0
    for i in range(0, n+1):
        result += i
    return result

# 시간 재기: 일반
start = perf_counter()
pure_sum(100000000)
print(perf_counter() - start)

# 시간 재기에 앞서 먼저 Compile을 해준다.
numba_sum(1)

# 시간 재기: Numba
start = perf_counter()
numba_sum(100000000)
print(perf_counter() - start)

- @jit 데커레이터의 모드
    - @jit 데커레이터는 nopython과 object라는 2가지 compilation 모드로 작동한다.
    - 위 예제에서 nopython=True를 통해 Numba에게 nopython 모드로 작동하라고 지시한 셈인데, 이 모드는 decorate된 function을 근본적으로 compile하여 Python Interpreter의 개입 없이 전체가 작동하도록 한다.
    - 만약 nopython 모드가 잘 작동하지 않을 경우, Numba은 object 모드를 통해 compile 할 수 있다. @jit(nopython=True)가 아닌 @jit이라고만 데커레이팅하면 이 모드가 작동하게 된다.
    - 본 모드에서는 Numba은 loop를 식별하여 machine code에서 compile하며 나머지는 Intereter code에서 compile하게 된다. 더 나은 성능을 기대한다면 이 모드가 아닌 nopython 모드를 사용해야 한다.
    

- 참고 : Polars(https://realpython.com/polars-python/)
- 참고 : Dask(https://data-science-note.tistory.com/67)